In [10]:
start_date = '2016-01-01'
end_date = '2021-06-01'
target = "5d_ret_standscalar"
alpha_name = "AlphaNetV1"
shift = 5
sequence = 6
LR = [0.001,0.0001,0.00001]
epoch_num = [30,30,30]

In [11]:
alpha_list = [
'open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover',
#             'open_ts_norm_30d',
#              'high_ts_norm_30d',
#              'low_ts_norm_30d',
#              'close_ts_norm_30d',
#              'vwap_ts_norm_30d',
#              'volume_ts_norm_30d',
#              'pct_chg_ts_norm_30d',
#              'turnover_ts_norm_30d',
#              'free_turnover_ts_norm_30d'
             ]
feat_list = alpha_list.copy()
method = ["COV","CORR","STD","ZSCORE","RETURN","DECAY"]
from AlphaNet.Data import generate_alpha_list
alpha_list = generate_alpha_list(feat_list,method,day=10)
alpha_list
# alpha_list = [i +"_residual" for i in feat_list]

['COV_open_high_10',
 'COV_open_low_10',
 'COV_open_close_10',
 'COV_open_vwap_10',
 'COV_open_volume_10',
 'COV_open_pct_chg_10',
 'COV_open_turnover_10',
 'COV_open_free_turnover_10',
 'COV_high_low_10',
 'COV_high_close_10',
 'COV_high_vwap_10',
 'COV_high_volume_10',
 'COV_high_pct_chg_10',
 'COV_high_turnover_10',
 'COV_high_free_turnover_10',
 'COV_low_close_10',
 'COV_low_vwap_10',
 'COV_low_volume_10',
 'COV_low_pct_chg_10',
 'COV_low_turnover_10',
 'COV_low_free_turnover_10',
 'COV_close_vwap_10',
 'COV_close_volume_10',
 'COV_close_pct_chg_10',
 'COV_close_turnover_10',
 'COV_close_free_turnover_10',
 'COV_vwap_volume_10',
 'COV_vwap_pct_chg_10',
 'COV_vwap_turnover_10',
 'COV_vwap_free_turnover_10',
 'COV_volume_pct_chg_10',
 'COV_volume_turnover_10',
 'COV_volume_free_turnover_10',
 'COV_pct_chg_turnover_10',
 'COV_pct_chg_free_turnover_10',
 'COV_turnover_free_turnover_10',
 'CORR_open_high_10',
 'CORR_open_low_10',
 'CORR_open_close_10',
 'CORR_open_vwap_10',
 'CORR_open_

In [2]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
from AlphaNet.Data import concat_original_data , generate_shift_data

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template


In [6]:
concat_original_data(alpha_name=alpha_name,alpha_list=alpha_list)

Loading the configuration from /home/ShareFolder/lgc/Modules/Research/config/feature_bt_template
['COV_open_high_10', 'COV_open_low_10', 'COV_open_close_10', 'COV_open_vwap_10', 'COV_open_volume_10', 'COV_open_pct_chg_10', 'COV_open_turnover_10', 'COV_open_free_turnover_10', 'COV_high_low_10', 'COV_high_close_10', 'COV_high_vwap_10', 'COV_high_volume_10', 'COV_high_pct_chg_10', 'COV_high_turnover_10', 'COV_high_free_turnover_10', 'COV_low_close_10', 'COV_low_vwap_10', 'COV_low_volume_10', 'COV_low_pct_chg_10', 'COV_low_turnover_10', 'COV_low_free_turnover_10', 'COV_close_vwap_10', 'COV_close_volume_10', 'COV_close_pct_chg_10', 'COV_close_turnover_10', 'COV_close_free_turnover_10', 'COV_vwap_volume_10', 'COV_vwap_pct_chg_10', 'COV_vwap_turnover_10', 'COV_vwap_free_turnover_10', 'COV_volume_pct_chg_10', 'COV_volume_turnover_10', 'COV_volume_free_turnover_10', 'COV_pct_chg_turnover_10', 'COV_pct_chg_free_turnover_10', 'COV_turnover_free_turnover_10', 'CORR_open_high_10', 'CORR_open_low_10

100%|██████████| 1305/1305 [01:14<00:00, 17.56it/s]


COV_open_high_10  COV_open_low_10  \
timestamp           ticker                                      
2016-01-28 09:30:00 600652      5.330995e+07     7.168249e+07   
                    600651      1.138230e+07     1.722683e+07   
                    600653      1.865701e+06     2.281064e+06   
                    600601      8.540686e+05     1.192663e+06   
                    600620      7.633118e+04     8.513148e+04   
...                                      ...              ...   
2021-05-31 09:30:00 300479     -1.582789e-01     3.603300e-01   
                    300162     -3.072889e-01     1.562311e+00   
                    300257     -4.639389e-01     3.559606e+00   
                    300196     -1.351096e+00     1.411886e+00   
                    688027     -2.035179e+00     1.158388e+01   

                            COV_open_close_10  COV_open_vwap_10  \
timestamp           ticker                                        
2016-01-28 09:30:00 600652       4.484323e+07      6.055622e+07   
                    600651       6.769108e+06      1.298848e+07   
                    600653       1.741328e+06      2.087812e+06   
                    600601       8.422265e+05      1.016886e+06   
                    600620       6.157798e+04      8.296504e+04   
...                                       ...               ...   
2021-05-31 09:30:00 300479      -1.726811e-01      1.494275e-02   
                    300162      -9.619111e-01     -2.989130e-01   
                    300257       1.627681e+00      1.208264e+00   
                    300196       6.102028e-01      4.299688e-01   
                    688027      -8.507000e-02      3.774474e+00   

                            COV_open_volume_10  COV_open_pct_chg_10  \
timestamp           ticker                                            
2016-01-28 09:30:00 600652        6.677806e+08        -35969.648438   
                    600651        2.213657e+08        -14329.779297   
                    600653       -1.574496e+07          -857.066040   
                    600601       -3.486446e+06          -406.247253   
                    600620        1.609142e+06          -369.932770   
...                                        ...                  ...   
2021-05-31 09:30:00 300479       -1.032120e+04            -2.834555   
                    300162       -1.017681e+05           -10.117500   
                    300257        3.280017e+03            -3.840266   
                    300196       -3.495013e+04            -3.008261   
                    688027       -5.191738e+03            -8.622447   

                            COV_open_turnover_10  COV_open_free_turnover_10  \
timestamp           ticker                                                    
2016-01-28 09:30:00 600652          10828.756836               11988.934570   
                    600651           2880.024414                3688.259277   
                    600653            -90.156387                -104.972328   
                    600601            -15.882930                 -17.963676   
                    600620             35.154175                  49.540543   
...                                          ...                        ...   
2021-05-31 09:30:00 300479             -0.614679                  -0.934943   
                    300162             -3.851849                  -5.317598   
                    300257              0.040098                   0.504496   
                    300196             -1.421734                  -1.816416   
                    688027             -3.054005                  -3.054005   

                            COV_high_low_10  COV_high_close_10  ...  \
timestamp           ticker                                      ...   
2016-01-28 09:30:00 600652     4.542331e+07       3.288108e+07  ...   
                    600651     1.195227e+07       1.058984e+07  ...   
                    600653     2.877538e+06       2.879107e+06  ...   
   

# Shift

In [4]:
generate_shift_data(alpha_name=alpha_name,shift=shift,sequence=sequence,target=target)

100%|██████████| 1705/1705 [00:26<00:00, 64.21it/s] 


[25, 20, 15, 10, 5, 0]


100%|██████████| 1280/1280 [06:36<00:00,  3.23it/s]


COV_open_high_10_Shift_25  \
timestamp           ticker                              
2016-02-29 09:30:00 000001                1243.930908   
                    000004                 398.704407   
                    000005                  67.774879   
                    000006                 995.594238   
                    000008                 289.304535   
...                                               ...   
2021-05-31 09:30:00 688698                   0.437478   
                    688777                   2.396356   
                    688788                  20.688845   
                    688819                   2.912261   
                    688981                   0.547056   

                            COV_open_low_10_Shift_25  \
timestamp           ticker                             
2016-02-29 09:30:00 000001                881.515686   
                    000004                353.650116   
                    000005                 59.929752   
                    000006                823.402954   
                    000008                309.922546   
...                                              ...   
2021-05-31 09:30:00 688698                  0.415361   
                    688777                  1.330733   
                    688788                 16.550266   
                    688819                  2.419755   
                    688981                  0.497840   

                            COV_open_close_10_Shift_25  \
timestamp           ticker                               
2016-02-29 09:30:00 000001                  842.104675   
                    000004                  311.025787   
                    000005                   53.047314   
                    000006                  712.251160   
                    000008                  200.656555   
...                                                ...   
2021-05-31 09:30:00 688698                    0.282948   
                    688777                    1.531856   
                    688788                   15.286644   
                    688819                    2.248206   
                    688981                    0.343938   

                            COV_open_vwap_10_Shift_25  \
timestamp           ticker                              
2016-02-29 09:30:00 000001                1125.538818   
                    000004                 342.218689   
                    000005                  61.585964   
                    000006                 880.694031   
                    000008                 293.617706   
...                                               ...   
2021-05-31 09:30:00 688698                   0.377242   
                    688777                   2.047794   
                    688788                  18.734159   
                    688819                   2.621625   
                    688981                   0.498400   

                            COV_open_volume_10_Shift_25  \
timestamp           ticker                                
2016-02-29 09:30:00 000001               -724324.250000   
                    000004                -25838.275391   
                    000005                136247.562500   
                    000006                300316.468750   
                    000008               -757594.812500   
...                                                 ...   
2021-05-31 09:30:00 688698                  2000.767700   
                    688777                  2879.518066   
                    688788                  8492.858398   
                    688819                  9878.734375   
                    688981                 13990.146484   

                            COV_open_pct_chg_10_Shift_25  \
timestamp           ticker                                 
2016-02-29 09:30:00 000001                    -39.612942   
                    000004                    -52.172729   
                    000005                    -22.81

In [5]:
generate_shift_data(alpha_name=alpha_name,shift=shift,sequence=sequence,target="5d_ret_standscalar")

100%|██████████| 1705/1705 [00:29<00:00, 58.31it/s] 


[25, 20, 15, 10, 5, 0]


100%|██████████| 1280/1280 [05:18<00:00,  4.02it/s]


COV_open_high_10_Shift_25  \
timestamp           ticker                              
2016-02-29 09:30:00 300001                 452.506653   
                    300002                 324.939178   
                    300004                 145.462341   
                    300005                 295.628052   
                    300006                  47.826797   
...                                               ...   
2021-05-31 09:30:00 688698                   0.437478   
                    688777                   2.396356   
                    688788                  20.688845   
                    688819                   2.912261   
                    688981                   0.547056   

                            COV_open_low_10_Shift_25  \
timestamp           ticker                             
2016-02-29 09:30:00 300001                414.008484   
                    300002                274.056274   
                    300004                132.233200   
                    300005                275.429688   
                    300006                 44.665237   
...                                              ...   
2021-05-31 09:30:00 688698                  0.415361   
                    688777                  1.330733   
                    688788                 16.550266   
                    688819                  2.419755   
                    688981                  0.497840   

                            COV_open_close_10_Shift_25  \
timestamp           ticker                               
2016-02-29 09:30:00 300001                  346.129883   
                    300002                  241.685089   
                    300004                  119.727837   
                    300005                  234.244934   
                    300006                   36.217121   
...                                                ...   
2021-05-31 09:30:00 688698                    0.282948   
                    688777                    1.531856   
                    688788                   15.286644   
                    688819                    2.248206   
                    688981                    0.343938   

                            COV_open_vwap_10_Shift_25  \
timestamp           ticker                              
2016-02-29 09:30:00 300001                 419.209351   
                    300002                 293.188904   
                    300004                 132.662933   
                    300005                 273.030243   
                    300006                  40.395844   
...                                               ...   
2021-05-31 09:30:00 688698                   0.377242   
                    688777                   2.047794   
                    688788                  18.734159   
                    688819                   2.621625   
                    688981                   0.498400   

                            COV_open_volume_10_Shift_25  \
timestamp           ticker                                
2016-02-29 09:30:00 300001               -592069.875000   
                    300002                 46865.171875   
                    300004                -41833.511719   
                    300005                -72575.929688   
                    300006                -22210.078125   
...                                                 ...   
2021-05-31 09:30:00 688698                  2000.767700   
                    688777                  2879.518066   
                    688788                  8492.858398   
                    688819                  9878.734375   
                    688981                 13990.146484   

                            COV_open_pct_chg_10_Shift_25  \
timestamp           ticker                                 
2016-02-29 09:30:00 300001                    -82.459778   
                    300002                    -53.137283   
                    300004                    -39.06

# Task Generation

In [12]:
task_info = np.load("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",allow_pickle=True).item()
original = task_info["Task"]
display(original)
# Timelist
time_list = ["2019-01-01","2019-06-01","2020-01-01","2020-06-01","2021-01-01","2021-06-01"]
time_list2 = []
for i in range(len(time_list)-1):
    time_list2.append([time_list[i],time_list[i+1]])
time_list2 = pd.DataFrame(time_list2,columns=["start_date","end_date"])
display(time_list2)

,Alpha_Name,sequence,LR,epoch_num,feature_num,start_date,end_date,status,description
task_id,,,,,,,,,
0,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2019-01-01,2019-06-01,finished,None
1,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2019-06-01,2020-01-01,finished,None
2,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2020-01-01,2020-06-01,finished,None
3,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2020-06-01,2021-01-01,finished,None
4,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2021-01-01,2021-06-01,finished,None
...,...,...,...,...,...,...,...,...,...
100,AlphaNetV1_Shift_5_Sequence_6_1d_ret_standscalar,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-01-01,2019-06-01,waiting,"{'target': '1d_ret_standscalar', 'alpha_name':..."
101,AlphaNetV1_Shift_5_Sequence_6_1d_ret_standscalar,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-06-01,2020-01-01,waiting,"{'target': '1d_ret_standscalar', 'alpha_name':..."
102,AlphaNetV1_Shift_5_Sequence_6_1d_ret_standscalar,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-01-01,2020-06-01,waiting,"{'target': '1d_ret_standscalar', 'alpha_name':..."


,start_date,end_date
0,2019-01-01,2019-06-01
1,2019-06-01,2020-01-01
2,2020-01-01,2020-06-01
3,2020-06-01,2021-01-01
4,2021-01-01,2021-06-01


In [13]:
task = pd.DataFrame([
                    alpha_name + "_Shift_%i_Sequence_%i_%s" % (shift,sequence,target),
                    sequence,LR,epoch_num,len(alpha_list),
                        ],index=["Alpha_Name","sequence","LR","epoch_num","feature_num"]).T
task['value']=1
time_list2['value']=1
task = pd.merge(task,time_list2,how='left',on='value')
del task['value']
task["status"] = "waiting"
task["description"] = [{
    "target" : target,
    "alpha_name" : alpha_name,
    "alpha_list" : alpha_list,
    "shift" : shift,
    "sequence" : sequence
} for i in range(len(task))]
task

,Alpha_Name,sequence,LR,epoch_num,feature_num,start_date,end_date,status,description
0,AlphaNetV1_Shift_5_Sequence_6_5d_ret_standscalar,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-01-01,2019-06-01,waiting,"{'target': '5d_ret_standscalar', 'alpha_name':..."
1,AlphaNetV1_Shift_5_Sequence_6_5d_ret_standscalar,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-06-01,2020-01-01,waiting,"{'target': '5d_ret_standscalar', 'alpha_name':..."
2,AlphaNetV1_Shift_5_Sequence_6_5d_ret_standscalar,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-01-01,2020-06-01,waiting,"{'target': '5d_ret_standscalar', 'alpha_name':..."
3,AlphaNetV1_Shift_5_Sequence_6_5d_ret_standscalar,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-06-01,2021-01-01,waiting,"{'target': '5d_ret_standscalar', 'alpha_name':..."
4,AlphaNetV1_Shift_5_Sequence_6_5d_ret_standscalar,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2021-01-01,2021-06-01,waiting,"{'target': '5d_ret_standscalar', 'alpha_name':..."


In [14]:
task = pd.concat([original,task],axis=0)
task.reset_index(drop=True,inplace=True)
task.index.names = ["task_id"]
task_info["Task"] = task
task[-20:]

,Alpha_Name,sequence,LR,epoch_num,feature_num,start_date,end_date,status,description
task_id,,,,,,,,,
90,AlphaNet_Moving5_Shift_3_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-01-01,2019-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
91,AlphaNet_Moving5_Shift_3_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-06-01,2020-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
92,AlphaNet_Moving5_Shift_3_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-01-01,2020-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
93,AlphaNet_Moving5_Shift_3_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-06-01,2021-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
94,AlphaNet_Moving5_Shift_3_Sequence_10,10,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2021-01-01,2021-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
95,AlphaNetV1_TsNorm30_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-01-01,2019-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
96,AlphaNetV1_TsNorm30_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2019-06-01,2020-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
97,AlphaNetV1_TsNorm30_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-01-01,2020-06-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."
98,AlphaNetV1_TsNorm30_Shift_5_Sequence_6,6,"[0.001, 0.0001, 1e-05]","[30, 30, 30]",108,2020-06-01,2021-01-01,waiting,"{'target': '10d_ret_standscalar', 'alpha_name'..."


In [15]:
np.save("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",task_info)
np.save("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/Task.npy",task_info)